<b><font size="5"><p>Inference - Model Deployment on Google Cloud Platform</p></b>


In [35]:
!pip install -U scikit-learn==0.20.4


  Using cached https://files.pythonhosted.org/packages/69/ee/46c86104541f18706642cf9ae32b5f7614b59cd024ec3a13b0c08751eba6/scikit_learn-0.20.4-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
tensorlayer 2.1.0 has requirement numpy<1.17,>=1.16, but you'll have numpy 1.18.1 which is incompatible.
tensorlayer 2.1.0 has requirement scikit-learn==0.21.0, but you'll have scikit-learn 0.20.4 which is incompatible.
tensorlayer 2.1.0 has requirement scipy==1.2.1, but you'll have scipy 1.4.1 which is incompatible.
imbalanced-learn 0.6.2 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.20.4 which is incompatible.
autogluon 0.0.5 has requirement scikit-learn==0.21.2, but you'll have scikit-learn 0.20.4 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [18]:
!python --version
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))


Python 3.7.3
The scikit-learn version is 0.20.4.


In [8]:
df = df_train_spark2.toPandas()

In [9]:
# This is done to create sklearn model because GCP does not yet support AutoGluon
X_train = df[FEATURES]
y_train = df[LABEL]


In [ ]:
#UnderSampling
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)

X, y = rus.fit_resample(X_train, y_train)

print(sorted(Counter(y).items()))


In [12]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression(random_state=0).fit(X_train, y_train)


/Users/jatinmalhotra/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
'''
from sklearn.ensemble import RandomForestClassifier

classifier=RandomForestClassifier(max_depth = 4,
                            n_estimators=100,
                            random_state = 42
                            ).fit(X, y)

'''

In [13]:
import pickle
pickle.dump(classifier, open('model.pkl', 'wb'))


<b><font size="3"><p>3.1 - Copy model.pkl file on google cloud storage </p></b>
    

In [14]:
!gsutil cp model.pkl gs://n-models/model.pkl    

Copying file://model.pkl [Content-Type=application/octet-stream]...
/ [1 files][  800.0 B/  800.0 B]                                                
Operation completed over 1 objects/800.0 B.                                      


<b><font size="3"><p>3.2 - Set GCP Project to be used for hosting a model</p></b>
    

In [15]:
!gcloud config set project "nest-ml"

Updated property [core/project].


<b><font size="3"><p>3.3 - Setup a new Model on GCP AI Platform</p></b>
    

In [16]:
!gcloud ai-platform models create "model_deployment11" --regions us-central1

Created ml engine model [projects/nest-ml/models/model_deployment11].


<b><font size="3"><p>3.4 - Set Environment Variables</p></b>

In [17]:
MODEL_DIR="gs://n-models"
VERSION_NAME="initial"
MODEL_NAME="model_deployment11"
FRAMEWORK="scikit-learn"

<b><font size="3"><p>3.5 -  Deploy</p></b>

In [ ]:
!gcloud ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.15 \
  --framework $FRAMEWORK \
  --python-version=3.7\
  --description="testing"


<b><font size="3"><p>3.6 -  Describe deployed Model</p></b>

In [ ]:
# Get details about deployed model
!gcloud ai-platform versions describe $VERSION_NAME \
  --model $MODEL_NAME

<b><font size="3"><p>3.7 -  Call Deloyed Model to make Inference on new data </p></b>

In [ ]:
# inference is a file containing testing data
!gcloud ai-platform predict --model "model_deployment" \
  --version "initial" \
  --json-instances "gcp_inherence_data.txt"

<b><font size="5"><p>THANK YOU </p></b>